In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from collections import deque

# RQ1

In [2]:
df = pd.read_csv('wiki-topcats-reduced.txt', sep=r'\t', header=None,  names=["start_vertex", "end_vertex"])

In [3]:
df.head()

,start_vertex,end_vertex
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


__Number of Edges__

In [4]:
edges=df.shape
num_of_edges = edges[0]
num_of_edges

2645247

__Number of Nodes__

In [5]:
source = set(df['start_vertex'].values)
destination = set(df['end_vertex'].values)

In [6]:
print(len(source))
print(len(destination))

428957
352518


In [7]:
Intersection_ = source.intersection(destination)
union_ = source.union(destination)
nodes = union_ - Intersection_
num_of_nodes = len(nodes)
num_of_nodes

140911

__Checking to see if our graph is directed?__

In [8]:
out_Link = df.groupby(['start_vertex'])['end_vertex'].apply(list).to_dict() #{54:[123, 124]} from 54 to 123 and 124
#{k: list(v) for k, v in df.groupby('Column1')['Column3']}

In [9]:
in_Link = df.groupby(['end_vertex'])['start_vertex'].apply(list).to_dict()
#{k: list(v) for k, v in df.groupby('Column1')['Column3']}

In [10]:
intersect_ = {}
intersect = 0
for item in out_Link.keys(  ):
    if item in in_Link.keys():
        intersect = set(out_Link[item]).intersection(set(in_Link[item]))
        if len(intersect)>0:   
            intersect_[item] = set(out_Link[item]).intersection(set(in_Link[item]))
print(len(intersect_))

249369


In [11]:
for item in out_Link.keys():
    
    for elem in set(out_Link[item]):
        
        if item not in set(in_Link[elem]):
            resul = 'directed'
            print(resul)
            break

# so here we didn't receive any output means 
# that our graph is not directed FOR REAL (WORKS 20 MIN)

__The average node degree__

In [26]:
# calculate degree of every node in the graph,
# sum them and divide by the number of nodes
# not nessesery as we can take just number of edges
# as our graph is not directed
sum_degree_out = 0
for node in out_Link.keys():
    node_degree = len(set(out_Link[node]))
    sum_degree_out = sum_degree_out + node_degree
sum_degree_in = 0
for node in in_Link.keys():
    node_degree = len(set(in_Link[node]))
    sum_degree_in = sum_degree_in + node_degree
average_degree = (sum_degree_out+sum_degree_in) / num_of_nodes 
average_degree   

37.544932617041965

In [30]:
average_degree = (2*(num_of_edges))//num_of_nodes
print(average_degree)

37


__Checking if our graph is a dense graph?__

In [27]:
D = 2 * num_of_edges/(num_of_nodes*(num_of_nodes-1))
D
# if D is close to 1 it means that graph is dense
# formula from wiki for undirected graphs
# D = 2*|E| / |V|*(|V|-1)
# so our graph is not dense

0.0002664461898874598

# RQ2


__Read page-names.txt__

In [28]:
df1 = pd.read_csv('wiki-topcats-page-names.txt', sep="\t", header=None,  names=["page_name"])

In [29]:
df1.head()

,page_name
0,0 Chiasmal syndrome
1,1 Kleroterion
2,2 Pinakion
3,3 LyndonHochschildSerre spectral sequence
4,4 Zariski's main theorem


In [30]:
df1['page_name'] = df1['page_name'].apply(lambda x: x[2:])
df1.head()

,page_name
0,Chiasmal syndrome
1,Kleroterion
2,Pinakion
3,LyndonHochschildSerre spectral sequence
4,Zariski's main theorem


In [31]:
df1['page_number'] = df1.index
df1.head()

,page_name,page_number
0,Chiasmal syndrome,0
1,Kleroterion,1
2,Pinakion,2
3,LyndonHochschildSerre spectral sequence,3
4,Zariski's main theorem,4


In [32]:
type(df1['page_name'])

pandas.core.series.Series

__Categories data set__

In [33]:
categories = {}
with open('wiki-topcats-categories.txt') as file:
    for list_ in file:
        if len(list_.split()[1:])>3500:
            categories[list_.split()[0][9:-1]] = list_.split()[1:]

In [34]:
categories.keys()

dict_keys(['English_footballers', 'The_Football_League_players', 'Association_football_forwards', 'Association_football_goalkeepers', 'Association_football_midfielders', 'Association_football_defenders', 'Living_people', 'Year_of_birth_unknown', 'Harvard_University_alumni', 'Major_League_Baseball_pitchers', 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 'Indian_films', 'Year_of_death_missing', 'English_cricketers', 'Year_of_birth_missing_(living_people)', 'Rivers_of_Romania', 'Main_Belt_asteroids', 'Asteroids_named_for_people', 'English-language_albums', 'English_television_actors', 'British_films', 'English-language_films', 'American_films', 'Fellows_of_the_Royal_Society', 'People_from_New_York_City', 'American_Jews', 'American_television_actors', 'American_film_actors', 'Debut_albums', 'Black-and-white_films', 'Year_of_birth_missing', 'Place_of_birth_missing_(living_people)', 'Article_Feedback_Pilot', 'American_military_personnel_of_World_War_II', 'Windows_gam

## Finding the shortest path

Given that the distance is the number of hops, and is optimal (shortest path.) We may keep track of visited nodes and current reachable nodes using Python's list/set. Starts from the first node and then keep hopping from the current set of nodes until we reach the target.
The point of visited-node list is to prevent visiting the visited nodes, resulting in a loop. And to get shortest distance, it's no use to make a revisit as it always makes the distance of resulting path longer.
This is a simple implementation of Breadth-first search. The efficiency partly depends on how to check visited nodes, and how to query adjacent nodes of given node. The Breadth-first search always guarantee to give optimal distance.

In [21]:
def bfs2(graph, start, goal):
    """
    finds a shortest path in directed `graph` between `start` and `goal`. 
    If no path is found, returns `None`
    """
    if start == goal:
        return [start]
    visited = {start}
    queue = deque([(start, [])])

    while queue:
        current, path = queue.popleft()
        visited.add(current)
        for neighbor in graph[current]:
            if neighbor == goal:
                return path + [current, neighbor]
            if neighbor in visited:
                continue
            queue.append((neighbor, path + [current]))
            visited.add(neighbor)   
    return None  # no path found. not strictly needed

In [23]:
if __name__ == '__main__':
    graph = {
        'A': set(['B', 'C']),  
        'B': set(['A', 'D', 'E']),
        'C': set(['A', 'F']),   
        'D': set(['B']),     
        'E': set(['B', 'F']),             
        'F': set(['C', 'E']),
    }
    path = bfs2(graph, 'D', 'F')
    if path:
        print(path)
    else:
        print('no path found')

['D', 'B', 'E', 'F']


In [24]:
L_=[]
for i in categories.keys():
    L_.append(len(categories[i]))

print(min(L_))
print(max(L_))
len(categories)

3501
418223


35

we have 35 categories. By random, choose one category as input category C0. Then compare c1 to c34 to C0. 
By comparing we mean calculating this formula: 

In [25]:
C0 = categories['English_footballers']
cat_name = []
for key in categories.keys():
    cat_name.append(key)
    
cat_name=cat_name[1:]  
len(cat_name)

34

In [26]:
cat_values = []
for name in cat_name:
    cat_values.append(categories[name])
    

In [27]:
len(cat_values)

34

In [28]:
shortest_path = {}
for category in cat_values:
    for start in C0:
        for end in category:
            
            path = bfs2(out_Link, start, end)
            shortest_path = len(path)-1           

KeyError: '22860'